In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import tensorflow as tf
#import  format_output
from datetime_extension import timedate_process
from produce_rnn_flow import aggregate_time
from math import sqrt,floor

In [5]:
#read block data
df=pd.read_csv("blocks.csv",header=0)
#print(df.shape)
df=timedate_process(df)
df=aggregate_time(df)
df.head()


,Level,latBlock,lngBlock,month,day,h14,h22,h9,h8,h19,...,h15_neighbor2,h16_neighbor2,h17_neighbor2,h18_neighbor2,h19_neighbor2,h20_neighbor2,h21_neighbor2,h22_neighbor2,h23_neighbor2,days
0,0,0,20,4,1,0,0,0,0,0,...,24.5,80.0,20.0,3.0,1.0,1.0,1.0,1.0,0.0,0
1,0,0,20,4,2,0,0,0,0,0,...,16.0,16.5,35.5,2.5,1.5,1.5,2.0,1.5,0.5,1
2,0,0,20,4,3,0,0,0,0,0,...,34.0,4.5,7.5,8.5,1.5,1.5,1.5,1.5,0.5,2
3,0,0,20,4,5,0,0,0,0,0,...,6.5,30.5,29.5,0.0,4.5,44.5,21.5,14.5,31.5,4
4,0,0,20,4,6,0,0,0,0,0,...,16.5,28.5,9.0,12.5,5.5,9.0,1.5,1.5,0.5,5


In [7]:
#print(max(df[(level=='0')].latBlock))
#df=df[(df.Level==0) & (df.latBlock==0) & (df.lngBlock==20)]
df=df[(df.Level==0)]
df.drop(['Level','latBlock','lngBlock','month','day'], axis=1,inplace=True)
print(df)

       h14  h22  h9  h8  h19  h17  h13  h15  h10  h1  ...   h15_neighbor2  \
0        0    0   0   0    0    0    0    0    1   0  ...            24.5   
1        0    0   0   0    0    0    0    0    0   0  ...            16.0   
2        0    0   0   0    0    1    0    0    1   0  ...            34.0   
3        0    0   0   0    0    1    1    0    1   0  ...             6.5   
4        0    0   0   0    0    0    2    0    1   0  ...            16.5   
5        0    0   0   0    0    1    0    1    0   0  ...            10.0   
6        0    0   0   0    0    0    1    0    0   0  ...            19.0   
7        0    0   0   0    0    0    0    1    0   0  ...            17.0   
8        1    0   0   0    0    0    0    0    0   0  ...             4.0   
9        0    0   0   0    0    0    0    0    0   0  ...            10.0   
10       0    0   0   0    0    1    0    0    0   0  ...             9.5   
11       2    0   0   0    0    0    0    0    0   0  ...            17.5   

In [8]:
data=[]
for i in range(0,len(df)):
    data.append(df.iloc[i].tolist())
data=np.array(data)

In [9]:
# define parameter
rnn_unit = 10  # hidden layer units
input_size = 73
output_size = 24
lr = 0.0006  # learning rate
N = 500 #ecope
N_model = 50 #each N time save model
length = 0
# ——————————————————input data——————————————————————
length=range(len(data))
#print(data)
# get train set
def get_train_data(batch_size=85, time_step=1, train_begin=0, train_end=length):
    batch_index = []
    data_train = data[train_begin:train_end]
    normalized_train_data = data_train
    #print(normalized_train_data)#
    train_x, train_y = [], []  # train set
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)
        x = normalized_train_data[i:i + time_step, :73]
        y = normalized_train_data[i:i + time_step, 0:24]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data) - time_step))
    #print(train_y)
    return batch_index, train_x, train_y


# test set
def get_test_data(time_step=1, test_begin=0):
    data_test = data[test_begin:]
    mean = np.mean(data_test, axis=0)
    std = np.std(data_test, axis=0)

    normalized_test_data = data_test  # normalized
    size = (len(normalized_test_data) + time_step - 1) // time_step  # 
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :73]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 0:24]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i + 1) * time_step:, :73]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 0:24]).tolist())
    return mean, std, test_x, test_y




weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    #'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
    'out': tf.Variable(tf.random_normal([rnn_unit, output_size]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    #'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
    'out': tf.Variable(tf.constant(0.1, shape=[output_size, ]))
}



def lstm(X):
    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input = tf.reshape(X, [-1, input_size])  
    input_rnn = tf.matmul(input, w_in) + b_in
    input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit]) 
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_states = tf.nn.dynamic_rnn(cell, input_rnn, initial_state=init_state,
                                                 dtype=tf.float32)  
    output = tf.reshape(output_rnn, [-1, rnn_unit])  
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_states



def train_lstm(batch_size=2, time_step=1, train_begin=0, train_end=10):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(batch_size, time_step, train_begin, train_end)
    with tf.variable_scope("sec_lstm"):
        pred, _ = lstm(X)

    loss = tf.reduce_mean(tf.square(tf.reshape(pred, [-1,output_size]) - tf.reshape(Y, [-1,output_size])))
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
    with tf.Session() as sess:
        try:
            module_file = tf.train.latest_checkpoint('.')
            saver.restore(sess, module_file)
        except:
            sess.run(tf.global_variables_initializer())
        # 重复训练N次
        for i in range(N):
            for step in range(len(batch_index) - 1):
                _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x[batch_index[step]:batch_index[step + 1]],
                                                                 Y: train_y[batch_index[step]:batch_index[step + 1]]})
            #print(i, loss_)
            if i % N_model==0:
                print("save model：", saver.save(sess, '.\prediction.model', global_step=i))

train_lstm()






INFO:tensorflow:Restoring parameters from .\prediction.model-450
save model： .\prediction.model-0
save model： .\prediction.model-50
save model： .\prediction.model-100
save model： .\prediction.model-150
save model： .\prediction.model-200
save model： .\prediction.model-250
save model： .\prediction.model-300
save model： .\prediction.model-350
save model： .\prediction.model-400
save model： .\prediction.model-450


In [10]:
def prediction(time_step=1,time_select=0):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    mean, std, test_x, test_y = get_test_data(time_step)
    with tf.variable_scope("sec_lstm", reuse=True):
        pred, _ = lstm(X)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        module_file = tf.train.latest_checkpoint('.')
        saver.restore(sess, module_file)
        
        
        test_predict = []
        for step in range(len(test_x) - 1):
            prob = sess.run(pred, feed_dict={X: [test_x[step]]})
            predict = prob.reshape((-1,1))
            test_predict.extend(predict)
        predict_test=[]
        for i in range(len(test_predict)):
            if i%24==time_select:
                predict_test.append(test_predict[i])
        #print(predict_test)
        y_test=[]
        test_y=np.array(test_y).reshape(-1,1)
        for i in range(len(test_y)):
            if i%24==time_select:
                y_test.append(test_y[i])
        test_y=np.array(y_test)
        test_predict=np.array(predict_test)
        test_y=np.nan_to_num(test_y)
        test_predict=np.nan_to_num(test_predict)
        test_y_result=[]
        test_predict_result=[]
        for i in test_y:
            test_y_result.append(i[0])
        for i in test_predict:
            test_predict_result.append(i[0])    
        test_predict_result=np.array(test_predict_result)
        test_y_result=np.array(test_y_result)
        result_list=[]
        #print(test_predict)
        for j in range(0,floor(min(len(test_y_result),len(test_predict_result))/85)):
            target=[]
            prediction=[]
            target=test_y_result[j*85:(j+1)*85]
            prediction=test_predict_result[j*85:(j+1)*85]
            error = []
            #print(min(len(target),len(prediction)))
            for i in range(min(len(target),len(prediction))):
                error.append(target[i] - prediction[i])
            squaredError = []
            absError = []
            #print(error)
            for val in error:
                squaredError.append(val * val)#target-prediction之差平方 
                absError.append(abs(val))#误差绝对值
            #print(j)
            result_list.append(sqrt(sum(squaredError) / len(squaredError)))
        return result_list
        #print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE
        #plt.figure()
        #plt.plot(list(range(len(test_predict))), test_predict, color='b')
        #plt.plot(list(range(len(test_y))), test_y,  color='r')
        #plt.show()
RMSE_result=[]
for i in range(0,24):
    RMSE_result.append(prediction(1,i))

INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring

In [11]:
rmseBlockRulst=[]
avg=0
for j in range(0,len(RMSE_result[0])):
    avg=0
    for i in RMSE_result:
        avg=avg+i[j]
    rmseBlockRulst.append(avg/24)
print(rmseBlockRulst)
print(max(rmseBlockRulst))

[0.9158378887569261, 3.6946211184022157, 18.611845750936315, 25.411664780924067, 2.7686516448709995, 0.9034930772429822, 1.058685761557973, 3.892359497070737, 11.83602680119099, 22.867165642903203, 32.22615640367918, 9.417351237728862, 10.238382525381555, 0.8748481077727538, 0.7037558986535433, 0.818908662719453, 1.0224713994812902, 1.324278007004114, 4.695530407504963, 11.251803188517156, 22.637041516527145, 30.223489131966797, 15.360902887741586, 21.625657708911106, 25.344342793927183, 5.701234236192806, 4.787702088799102, 0.9756018135340462, 0.7179643207586551, 0.7230228935622912, 0.863490137414458, 0.9610940013168259, 0.9945769807224742, 1.3128946373105028, 3.1614344749996732, 8.766433214543435, 29.09331969386916, 25.34656100717807, 25.066639745418627, 27.72935917022271, 20.80277000376067, 23.83718272694507, 21.066529455275028, 26.9063697984901, 4.161870012511164, 3.1792824041970085, 0.8616652663823992, 0.7442125056524214, 0.6547415390948831, 0.690912197416855, 0.8348902511003121, 

In [17]:
array_rmse=np.array(rmseBlockRulst)
print(array_rmse.mean())

16.344747137810796
